<a href="https://colab.research.google.com/github/667029/dat158-ML-assignment2/blob/main/testDAT158ML2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [51]:
train_url = "https://raw.githubusercontent.com/667029/dat158-ML-assignment2/refs/heads/main/df_train.csv"
test_url = "https://raw.githubusercontent.com/667029/dat158-ML-assignment2/refs/heads/main/df_test.csv"

train_data = pd.read_csv(train_url)
test_data = pd.read_csv(test_url)

train_data = train_data.drop(columns=['date'])
test_data = test_data.drop(columns=['date'])

In [52]:
print("Treningsdata:")
print(train_data.head())
print("\nTestdata:")
print(test_data.head())

Treningsdata:
       price  bedrooms  grade  has_basement  living_in_m2  renovated  \
0   237000.0         3      7         False     120.77390      False   
1  1328000.0         5      9          True     310.29602       True   
2   790500.0         4      9         False     311.22505      False   
3   431000.0         4      9         False     282.42512      False   
4   199000.0         3      7          True     162.58025      False   

   nice_view  perfect_condition  real_bathrooms  has_lavatory  single_floor  \
0      False              False               1         False          True   
1       True              False               3         False          True   
2      False              False               2          True         False   
3      False              False               2          True         False   
4      False              False               2          True          True   

   month  quartile_zone  
0      6              2  
1      4              4  


In [53]:
print("\nManglende verdier i treningsdata:", train_data.isnull().sum())
print("Manglende verdier i testdata:", test_data.isnull().sum())


Manglende verdier i treningsdata: price                0
bedrooms             0
grade                0
has_basement         0
living_in_m2         0
renovated            0
nice_view            0
perfect_condition    0
real_bathrooms       0
has_lavatory         0
single_floor         0
month                0
quartile_zone        0
dtype: int64
Manglende verdier i testdata: price                0
bedrooms             0
grade                0
has_basement         0
living_in_m2         0
renovated            0
nice_view            0
perfect_condition    0
real_bathrooms       0
has_lavatory         0
single_floor         0
month                0
quartile_zone        0
dtype: int64


In [54]:
train_data = train_data.fillna(train_data.mean())
test_data = test_data.fillna(test_data.mean())

In [55]:
X_train = train_data.drop('price', axis=1)  # Alle kolonner unntatt 'price'
y_train = train_data['price']               # Målvariabelen 'price'

X_test = test_data.drop('price', axis=1)    # Testsett uten 'price'
y_test = test_data['price']                 # Målvariabelen for test

In [56]:
model = RandomForestRegressor(random_state=42)

In [ ]:
#model.fit(X_train, y_train)

In [80]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

In [49]:
#random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42)
#random_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_jobs=-1,
                   param_distributions={'max_depth': [10, 20],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [50, 100]},
                   random_state=42, scoring='neg_mean_absolute_error')

In [75]:
print("Beste parametere:", grid_search.best_params_)

Beste parametere: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}


In [76]:
best_model = grid_search.best_estimator_

In [77]:
y_pred = best_model.predict(X_test)

In [78]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [79]:
print("\nModellevaluering:")
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R²):", r2)


Modellevaluering:
Mean Absolute Error (MAE): 82946.84053971786
Mean Squared Error (MSE): 14113504533.618282
Root Mean Squared Error (RMSE): 118800.27160582707
R-squared (R²): 0.7676286283958531


In [72]:
importances = best_model.feature_importances_
feature_names = X_train.columns
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print("\nFeature Importance:")
print(feature_importances)


Feature Importance:
              Feature  Importance
11      quartile_zone    0.392146
1               grade    0.282513
3        living_in_m2    0.220637
5           nice_view    0.031499
10              month    0.016201
2        has_basement    0.011937
0            bedrooms    0.009708
7      real_bathrooms    0.008987
6   perfect_condition    0.008582
4           renovated    0.007502
9        single_floor    0.006797
8        has_lavatory    0.003489
